# Question 1: Structural Analysis of Political Power

**Student:** Erfan Shahabi (810103166)

In [ ]:
import os
import sys

# Add parent directory to path
notebook_dir = os.getcwd()
parent_dir = os.path.dirname(notebook_dir)
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

print(f"Working directory: {os.getcwd()}")
print(f"Parent directory added: {parent_dir}")

In [ ]:
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Direct imports from src
from src.question1 import centrality
from src.question1 import analysis
from src.question1 import bottlenecks
from src.question1 import efficiency
from src.question1 import bonacich
from src.question1 import visualization

print("All modules imported successfully!")

## Load Data

In [ ]:
# Load edge list
edges_df = pd.read_csv('../data/politician.edges.csv')
print(f"Edges: {len(edges_df)}")
print(edges_df.head())

# Load node attributes
nodes_df = pd.read_csv('../data/politician.nodes.csv')
print(f"\nNodes: {len(nodes_df)}")
print(nodes_df.head())

In [ ]:
# Create graph
G = nx.from_pandas_edgelist(edges_df, source='source', target='target')

# Add node attributes if available
if 'id' in nodes_df.columns:
    node_attrs = nodes_df.set_index('id').to_dict('index')
    nx.set_node_attributes(G, node_attrs)

print(f"Graph: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges")
print(f"Connected: {nx.is_connected(G)}")

## Part (a): Power Geometry

### 1. Centrality Calculations

In [ ]:
# Calculate all centralities
centralities = centrality.calculate_all_centralities(G)

# Create summary dataframe
summary_df = pd.DataFrame({
    'node': list(centralities['degree'].keys()),
    'degree': list(centralities['degree'].values()),
    'eigenvector': list(centralities['eigenvector'].values()),
    'closeness': list(centralities['closeness'].values())
})

summary_df['deg_rank'] = summary_df['degree'].rank(ascending=False)
summary_df['eig_rank'] = summary_df['eigenvector'].rank(ascending=False)
summary_df['close_rank'] = summary_df['closeness'].rank(ascending=False)

print("Top 10 by Degree:")
print(summary_df.nsmallest(10, 'deg_rank')[['node', 'degree', 'deg_rank']])

print("\nTop 10 by Eigenvector:")
print(summary_df.nsmallest(10, 'eig_rank')[['node', 'eigenvector', 'eig_rank']])

print("\nTop 10 by Closeness:")
print(summary_df.nsmallest(10, 'close_rank')[['node', 'closeness', 'close_rank']])

### 2. Gap Analysis

In [ ]:
gap_df = analysis.gap_analysis(centralities)

fig = visualization.plot_degree_eigenvector_scatter(gap_df)
plt.show()

print("\nNodes with significant deviation:")
print(gap_df.head(20))

### 3. Hub Analysis

In [ ]:
# Find anomalies: Low Degree but High Eigenvector
anomalies = analysis.identify_hub_anomalies(centralities, degree_threshold=100, eig_threshold=50)

print("Hub Anomalies (Low Degree, High Eigenvector):")
print(anomalies)

print("\nCloseness Analysis of Top 3 Anomalies:")
for idx, row in anomalies.head(3).iterrows():
    print(f"\nNode {row['node']}:")
    print(f"  Degree Rank: {row['degree_rank']:.0f}")
    print(f"  Eigenvector Rank: {row['eig_rank']:.0f}")
    print(f"  Closeness Rank: {row['close_rank']:.0f}")

## Part (b): Information Bottlenecks

In [ ]:
# Calculate betweenness
betweenness_df = bottlenecks.rank_gap_analysis(G, top_n=10)

print("Top 10 Betweenness Centrality:")
print(betweenness_df)

fig = visualization.plot_betweenness_ranking(betweenness_df, top_n=10)
plt.show()

## Part (c): Efficient Monitors

In [ ]:
# Identify efficient monitors
efficient_df = efficiency.identify_efficient_monitors(G, top_closeness=20, degree_threshold=100)

print("Efficient Monitors:")
print(efficient_df)

# Plot scatter
all_nodes_df = pd.DataFrame({
    'node': list(G.nodes()),
    'norm_degree': [centralities['degree'][n] for n in G.nodes()],
    'closeness': [centralities['closeness'][n] for n in G.nodes()]
})

fig = visualization.plot_efficiency_scatter(all_nodes_df)
plt.show()

In [ ]:
# Visualize ego networks
for idx, row in efficient_df.head(3).iterrows():
    node = row['node']
    ego_g = efficiency.extract_ego_network(G, node)
    
    print(f"\nEgo Network of {node}:")
    print(f"  Nodes: {ego_g.number_of_nodes()}")
    print(f"  Edges: {ego_g.number_of_edges()}")
    
    fig = visualization.plot_ego_network(G, ego_g, node, 
                                        save_path=f'../results/plots/ego_{node}.png')
    plt.show()

## Part (d): Bonacich Power

In [ ]:
# Analyze power regimes
beta_values = [0, 0.01, 0.02, -0.01, -0.02]
bonacich_df = bonacich.analyze_power_regimes(G, beta_values=beta_values)

print("Bonacich Power Rankings:")
print(bonacich_df.head(10))

# Plot trajectories
fig = visualization.plot_bonacich_trajectories(bonacich_df)
plt.show()

# Analyze rank changes
bonacich_df['rank_change_pos'] = bonacich_df['rank_0.02'] - bonacich_df['rank_0']
bonacich_df['rank_change_neg'] = bonacich_df['rank_-0.02'] - bonacich_df['rank_0']

print("\nPower Amplifiers:")
print(bonacich_df.nsmallest(5, 'rank_change_pos')[['rank_0', 'rank_0.02', 'rank_change_pos']])

print("\nPower Inhibitors:")
print(bonacich_df.nlargest(5, 'rank_change_pos')[['rank_0', 'rank_0.02', 'rank_change_pos']])